In [5]:
import requests
from bs4 import BeautifulSoup 
import re
import unicodedata
import pythainlp.util
from pythainlp.tokenize import word_tokenize
from pythainlp.util import find_keyword
from pythainlp.util import rank
#from pythainlp.summarize import extract_keywords
from pythainlp.summarize import summarize

from urllib.parse import urljoin

In [18]:
def get_all_links(url, depth=0, visited={}):
  headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}
  response = requests.get(url,headers=headers)
  soup = BeautifulSoup(response.text, 'html.parser')
  links = soup.find_all('a')
  links = [link.get('href') for link in links if link.get('href') and not link.get('href').startswith('#')]
  links = [link for link in links if link.startswith(url) or link.startswith('/')]
  links = [urljoin(url, link) for link in links if link]


  # Recursively crawl the links at the next depth level
  if depth < 3: #3
    new_links = []
    for link in links:
      # Increment the visit count for the link
      if link in visited:
        visited[link] += 1
      else:
        visited[link] = 1
      # Get the newly-crawled links and add them to the list
        new_links.extend(get_all_links(link, depth=depth+1, visited=visited))
      # Add the newly-crawled links to the original list
    links.extend(new_links)

  return visited

base_url = 'https://www.bbc.com/news'
website_dict = get_all_links(base_url, depth=0, visited={})
print(website_dict)



{'https://www.bbc.com/news': 60, 'https://www.bbc.com/news/world-60525350': 26, 'https://www.bbc.com/news/coronavirus': 31, 'https://www.bbc.com/news/science-environment-56837908': 26, 'https://www.bbc.com/news/av/10462520': 29, 'https://www.bbc.com/news/world': 38, 'https://www.bbc.com/news/world/asia': 57, 'https://www.bbc.com/news/uk': 41, 'https://www.bbc.com/news/business': 48, 'https://www.bbc.com/news/technology': 26, 'https://www.bbc.com/news/science_and_environment': 27, 'https://www.bbc.com/news/stories': 30, 'https://www.bbc.com/news/entertainment_and_arts': 26, 'https://www.bbc.com/news/health': 31, 'https://www.bbc.com/news/world_radio_and_tv': 27, 'https://www.bbc.com/news/in_pictures': 26, 'https://www.bbc.com/news/reality_check': 26, 'https://www.bbc.com/news/newsbeat': 26, 'https://www.bbc.com/news/the_reporters': 26, 'https://www.bbc.com/news/world/africa': 11, 'https://www.bbc.com/news/world/australia': 11, 'https://www.bbc.com/news/world/europe': 30, 'https://www.bb

In [ ]:
class Thai:
    def __init__(self,data:list):
        self.data_value = data
        self.sentence = self.get_sentence()
        self.keyword = self.get_keyword()
        self.summarize = self.get_summarize()
    def make_sentence(self,list_word):
        self.sentence_value = ''
        for i in list_word:
            for i in list_word:
                if pythainlp.util.countthai(i)<10:
                    list_word.remove(i)
        self.sentence_value = ' '.join(list_word)
        return self.sentence_value
    def get_sentence(self):
        self.sentence_result = self.make_sentence(self.data_value)
        return self.sentence_result
    def get_keyword(self):
        self.keyword_result = {}
        self.keyword_value = word_tokenize(self.sentence, engine="newmm")
        self.keyword_dict = find_keyword(self.keyword_value)
        # Iterate over the keys in the dictionary
        for key in self.keyword_dict:
        # Check if the key is text (i.e., not a space or quotation mark)
            if key.isalpha():
            # If the key is text, add it to the new dictionary
                self.keyword_result[key] = self.keyword_dict[key]
        return self.keyword_result
    def get_summarize(self):
        self.summarize_result =[]
        self.summarize_result = summarize(self.sentence,n=5)
        return self.summarize_result


In [25]:
def scrape_tags(url):
  # Make a request to the website
  response = requests.get(url)
  # Parse the HTML of the website
  soup = BeautifulSoup(response.text, 'html.parser')
  # Find all p tags and title tags
  title_tag = soup.find('title').text
  p_tags = soup.find_all('p')
  p_list =[]
  for p in p_tags:
    if p.string != None:
      p_list.append(unicodedata.normalize("NFKD", p.string))
  if len(p_list) == 0:
    p_list.append('ไม่พบข้อความในเว็บนี้')
  
  p_list = "".join(p_list)
  #p_list = Thai(p_list) 
  #p_list=p_list.summarize
  
  
  
  return p_list, title_tag

import sqlite3
conn = sqlite3.connect('scraped_data.db')
conn.execute('''CREATE TABLE DATA
             (ID INTEGER PRIMARY KEY AUTOINCREMENT,
             WEBSITE STRING NOT NULL,
             BODY TEXT NOT NULL,
             TITLE TEXT NOT NULL);''')
for website in website_dict.keys():
  if 'news' in website:
    print(p)
    p, title = scrape_tags(website)
    
  conn.execute("INSERT INTO DATA (WEBSITE, BODY, TITLE) VALUES (?, ?, ?)", (website, p[0], title))
  print(f'For website {website}\n the p tags is: {p} \n the title tag is: {title}')



conn.commit()
conn.close()

["More classified material found at Biden's home", 'Top secret documents reportedly found in Biden cache', 'Four unanswered questions about the Biden documents', 'How Biden and Trump secret files cases compare', 'Delay in telling public about files may haunt Biden', "Republicans want to know who visited Biden's homes", 'The problem with politicians and classified files', 'Biden under investigation', 'How much trouble is the president in over classified documents found at his home?', 'Who is the special counsel in the Biden probe?', 'Special counsel appointed to investigate Biden files', 'Second batch of classified Biden documents found', 'Jill Biden has surgery to remove cancerous skin lesions', "Biden 'surprised' about classified files discovery", 'Biden visits US-Mexico border in trip to Texas', 'Western allies to send fighting vehicles to Ukraine', 'What will change with Republicans controlling the House', "What's actually in the $1.7tn US spending bill?", "Zelensky trip shows US do